In [130]:
import numpy as np
import csv
from cqs.object import Instance
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit.random import random_circuit
import qiskit.qasm3 as qasm3
from instances_b.reader_b import read_csv_b
from transpiler.transpile import transpile_circuit
from examples.benchmark.cqs_simulation import main_prober, main_solver
from qiskit.quantum_info import Operator
from tqdm import tqdm

def normalize_quantum_state(vec):
    norm=np.sqrt(np.abs(vec.T.conjugate()@vec))
    return vec/norm

def __num_to_pauli_list(num_list):
    paulis = ['I', 'X', 'Y', 'Z']
    pauli_list = [paulis[int(i)] for i in num_list]
    return pauli_list

def __add_Pauli_gate(qc, which_qubit, which_gate):
    if which_gate == 0:
        qc.id(which_qubit)
    elif which_gate == 1:
        qc.x(which_qubit)
    elif which_gate == 2:
        qc.y(which_qubit)
    elif which_gate == 3:
        qc.z(which_qubit)
    else:
        return ValueError("Not supported Pauli gate type.")

def __num_to_pauli_circuit(num_list):
    n = len(num_list)
    num_list = [int(i) for i in num_list]
    qr = QuantumRegister(n, 'q')
    qc = QuantumCircuit(qr)
    for i in range(n):
        __add_Pauli_gate(qc, i, num_list[i])
    return qc

def create_random_circuit_in_native_gate(n, d):
    ub = random_circuit(num_qubits=n,max_operands=2, depth=d, measure=False)
    # ub = transpile_circuit(ub, device='Aria', optimization_level=2)
    return ub

In [46]:
with open('3_qubit_data_generation_matrix_A.csv', 'r', newline='') as csvfile:
    data_b = read_csv_b(3)
    reader = csv.reader(csvfile, delimiter=' ', quotechar='|')
    for i, row in enumerate(reader):
        if 2 > i > 0:
            row_clean = [j for j in ''.join(row).split('"') if j != ',']
            nLc = row_clean[0].split(',')
            n = int(nLc[0])
            print("qubit number is:", n)
            L = int(nLc[1])
            print("term number is:", L)
            kappa = float(nLc[2])
            print('condition number is', kappa)
            pauli_strings = [__num_to_pauli_list(l) for l in eval(row_clean[1])]
            print('Pauli strings are:', pauli_strings)
            pauli_circuits = [__num_to_pauli_circuit(l) for l in eval(row_clean[1])]
            coeffs = [float(i) for i in eval(row_clean[2])]
            print('coefficients are:', coeffs)
            print()

            # circuit depth d
            d = 3
            ub = qasm3.loads(data_b.iloc[i].qasm)
            print('Ub is given by:', data_b.iloc[i].b)
            print(ub)

            # generate instance
            instance = Instance(n, L, kappa)
            instance.generate(given_coeffs=coeffs, given_unitaries=pauli_circuits, given_ub=ub)
            Itr, LOSS, ansatz_tree = main_prober(instance, backend='qiskit-noiseless',ITR=None,
                                    shots=0, optimization_level=2,
                                    noise_level_two_qubit=0, noise_level_one_qubit=0, readout_error=0)
            print(Itr)
            print(LOSS)
            print('ANsatz tree contains:')
            # for qc in ansatz_tree:
            #     print(qc)

Resolved file path: C:\Users\Nikita\PycharmProjects\CQS_singapore\instances_b\3_b_random_circuits.csv
Reading file: C:\Users\Nikita\PycharmProjects\CQS_singapore\instances_b\3_b_random_circuits.csv
qubit number is: 3
term number is: 3
condition number is 3.5456037521362305
Pauli strings are: [['Y', 'Z', 'Y'], ['I', 'Y', 'Y'], ['X', 'Y', 'I']]
coefficients are: [-1.01, 1.17, 2.76]

Ub is given by: [(0.7071067811865475+0j), 0j, 0j, 0j, (-0.6099094210942649+0.357785547584665j), 0j, 0j, 0j]
                                           
q_0: ──────────X───────────■───────■───────
               │           │ ┌─────┴──────┐
q_1: ──────────X───────────■─┤ Rz(4.3897) ├
     ┌───────────────────┐   └────────────┘
q_2: ┤ U2(2.6111,5.8783) ├─────────────────
     └───────────────────┘                 


Hadamard tests Progress: 100%|██████████| 9/9 [00:00<00:00, 12.37it/s]


loss: 1.0
combination parameters are: [(8.875921976271895e-19+8.183583031656932e-43j)]


Hadamard tests Progress: 100%|██████████| 27/27 [00:02<00:00, 12.51it/s]


loss: 0.23874246416144873
combination parameters are: [(2.020011658596599e-18+1.2240697286717896e-17j), (0.27581796050071716-3.062195165274972e-17j)]


Hadamard tests Progress: 100%|██████████| 54/54 [00:05<00:00, 10.67it/s]


loss: 0.040361668973984965
combination parameters are: [(1.0263140879855734e-17-0.09446863830089569j), (0.27581796050071716-4.2345076239928896e-17j), (5.660836991786247e-17-0.16955626010894775j)]


Hadamard tests Progress: 100%|██████████| 90/90 [00:08<00:00, 10.99it/s]


loss: 6.770843941072968e-08
combination parameters are: [(-1.1195258858408969e-13-0.09446864575147629j), (0.3184279203414917-4.2979942521670367e-14j), (-1.1223352569577141e-13-0.16955624520778656j), (0.07647912949323654+1.0880992863725267e-14j)]
[1, 2, 3, 4]
[1.0, 0.23874246416144873, 0.040361668973984965, 6.770843941072968e-08]
ANsatz tree contains:


array([[ 0.  +0.j  ,  0.  +0.j  ,  0.  +0.j  ,  0.  -2.76j,  0.  +0.j  ,
         1.01+0.j  , -1.17+0.j  ,  0.  +0.j  ],
       [ 0.  +0.j  ,  0.  +0.j  ,  0.  -2.76j,  0.  +0.j  , -1.01+0.j  ,
         0.  +0.j  ,  0.  +0.j  , -1.17+0.j  ],
       [ 0.  +0.j  ,  0.  +2.76j,  0.  +0.j  ,  0.  +0.j  ,  1.17+0.j  ,
         0.  +0.j  ,  0.  +0.j  , -1.01+0.j  ],
       [ 0.  +2.76j,  0.  +0.j  ,  0.  +0.j  ,  0.  +0.j  ,  0.  +0.j  ,
         1.17+0.j  ,  1.01+0.j  ,  0.  +0.j  ],
       [ 0.  +0.j  , -1.01+0.j  ,  1.17+0.j  ,  0.  +0.j  ,  0.  +0.j  ,
         0.  +0.j  ,  0.  +0.j  ,  0.  -2.76j],
       [ 1.01+0.j  ,  0.  +0.j  ,  0.  +0.j  ,  1.17+0.j  ,  0.  +0.j  ,
         0.  +0.j  ,  0.  -2.76j,  0.  +0.j  ],
       [-1.17+0.j  ,  0.  +0.j  ,  0.  +0.j  ,  1.01+0.j  ,  0.  +0.j  ,
         0.  +2.76j,  0.  +0.j  ,  0.  +0.j  ],
       [ 0.  +0.j  , -1.17+0.j  , -1.01+0.j  ,  0.  +0.j  ,  0.  +2.76j,
         0.  +0.j  ,  0.  +0.j  ,  0.  +0.j  ]])

In [87]:
loss, alphas = main_solver(instance, ansatz_tree, backend='qiskit-noiseless', ITR=None, shots=0, optimization_level=2)

array([[ 0.        +6.19534276e-19j,  0.        -0.00000000e+00j,
         0.        +0.00000000e+00j,  0.        -5.27865968e-01j,
         0.        -0.00000000e+00j, -0.19316834-0.00000000e+00j,
         0.22376927+0.00000000e+00j,  0.        -0.00000000e+00j],
       [ 0.        -0.00000000e+00j,  0.        -6.19534276e-19j,
         0.        -5.27865968e-01j,  0.        -0.00000000e+00j,
         0.19316834+0.00000000e+00j,  0.        -0.00000000e+00j,
         0.        -0.00000000e+00j,  0.22376927+0.00000000e+00j],
       [ 0.        +0.00000000e+00j,  0.        +5.27865968e-01j,
        -0.        -4.26903841e-18j,  0.        +0.00000000e+00j,
        -0.22376927+0.00000000e+00j,  0.        +0.00000000e+00j,
         0.        +0.00000000e+00j,  0.19316834+0.00000000e+00j],
       [ 0.        +5.27865968e-01j,  0.        +0.00000000e+00j,
         0.        +0.00000000e+00j,  0.        +4.26903841e-18j,
         0.        +0.00000000e+00j, -0.22376927+0.00000000e+00j,
       

In [138]:
A_inv=np.linalg.inv(np.array(instance.get_matrix()))
b=np.array(Operator(instance.get_ub().to_gate())).T[0].reshape(-1,1)
solution=np.zeros(2**3,dtype=np.complex128)
for i in tqdm(range(len(ansatz_tree))):
    ub_copy=ub.copy()
    circuit_u_i=ub_copy.compose(ansatz_tree[i].to_gate(), list(range(ansatz_tree[i].num_qubits)))
    ui=np.array(Operator(circuit_u_i)).T[0]
    solution+=alphas[i] * ui
solution=normalize_quantum_state(solution.reshape(-1,1))
solution_true=A_inv@b
solution_true=normalize_quantum_state(solution_true)
fidelity=np.abs(solution.T.conjugate()@solution_true)
print("fidelity",fidelity)
print("loss",loss)


100%|██████████| 5/5 [00:00<00:00, 681.67it/s]

fidelity [[0.62925318]]
loss 6.966621490800406e-08


In [141]:
instance.get_matrix()

array([[ 0.  +0.j  ,  0.  +0.j  ,  0.  +0.j  ,  0.  -2.76j,  0.  +0.j  ,
         1.01+0.j  , -1.17+0.j  ,  0.  +0.j  ],
       [ 0.  +0.j  ,  0.  +0.j  ,  0.  -2.76j,  0.  +0.j  , -1.01+0.j  ,
         0.  +0.j  ,  0.  +0.j  , -1.17+0.j  ],
       [ 0.  +0.j  ,  0.  +2.76j,  0.  +0.j  ,  0.  +0.j  ,  1.17+0.j  ,
         0.  +0.j  ,  0.  +0.j  , -1.01+0.j  ],
       [ 0.  +2.76j,  0.  +0.j  ,  0.  +0.j  ,  0.  +0.j  ,  0.  +0.j  ,
         1.17+0.j  ,  1.01+0.j  ,  0.  +0.j  ],
       [ 0.  +0.j  , -1.01+0.j  ,  1.17+0.j  ,  0.  +0.j  ,  0.  +0.j  ,
         0.  +0.j  ,  0.  +0.j  ,  0.  -2.76j],
       [ 1.01+0.j  ,  0.  +0.j  ,  0.  +0.j  ,  1.17+0.j  ,  0.  +0.j  ,
         0.  +0.j  ,  0.  -2.76j,  0.  +0.j  ],
       [-1.17+0.j  ,  0.  +0.j  ,  0.  +0.j  ,  1.01+0.j  ,  0.  +0.j  ,
         0.  +2.76j,  0.  +0.j  ,  0.  +0.j  ],
       [ 0.  +0.j  , -1.17+0.j  , -1.01+0.j  ,  0.  +0.j  ,  0.  +2.76j,
         0.  +0.j  ,  0.  +0.j  ,  0.  +0.j  ]])

In [142]:
instance.get_coeffs()

[-1.01, 1.17, 2.76]